In [2]:
%load_ext autoreload
%autoreload 2

In [ ]:
from table_environment import PickPlaceEnv
from motion_primitives import Motion_primitives
from utils import (
    save_image,
    display_image,
    display_images,
    extract_action_details,
    is_task_successful,
    validate_action_execution,
    save_json,
    load_json,
    validate_plan,
    run_experiment,
    save_images,
    Replanner
)
from tasks import task1, task2, task3, task4
from custom_logger import FileLogger
import os
import re
import pandas as pd
import time

In [4]:
logger = FileLogger(log_filename="log.txt", log_dir=".", log_to_stdout=True)
logger.set_log_level("INFO")

In [ ]:
# ------------------------------------------------------------------------------------------------
# - Parameters
# ------------------------------------------------------------------------------------------------

experiment_name = "lera_test_replanner"
tasks = [task4]
model_name = "gpt-4o"
images_to_use = 4
images_step = 30
drop_prob = (0, 1.0)
do_replan = True
save_video = True


# -------------------------------------------------------------------------------------------------
# - Experiment
# -------------------------------------------------------------------------------------------------
logger.info(f"="*50)
logger.info(f"Running experiment: {experiment_name}")
logger.info(f"="*50)
base_path = f"~/work/pybullet/Pybullet/results_logs/"
experiment_path = os.path.join(base_path, experiment_name)
if not os.path.exists(experiment_path):
    os.makedirs(experiment_path)

replanner = Replanner(experiment_name, images_to_use=images_to_use, images_step=images_step, model_name=model_name)

for task in tasks:
    for exp_run in range(10):
        if os.path.exists(os.path.join(experiment_path, task["name"] + f"_{exp_run}.json")):
            logger.info(f"Skipping task: {task['name']} + _{exp_run}, log already exists")
            continue
        logger.info(f"-"*50)
        logger.info(f"Running task: {task['name']}, goal: {task['goal']}")
        logger.info(f"-"*50)
        goal = task["goal"]
        obj_list = task["obj_list"]
        plan = task["plan"]
        validation_rule = task["validation_rule"]
        
        env = PickPlaceEnv(render=True)
        _ = env.reset(obj_list)
        motion = Motion_primitives(env, obj_list, drop_type='place', drop_prob=drop_prob)
        env_actions = {
            'locate': motion.locate,
            'pick': motion.pick,
            'place': motion.place,
            'done': lambda args=None: is_task_successful(env, validation_rule)
        }  
        
        available_actions = list(set([
            f"{action}('{obj}')" if action != 'done' else "done()"
            for action in env_actions
            for obj in obj_list
        ]))
        
        result_states = run_experiment(
            env=env, 
            replanner=replanner,
            obj_list=obj_list, 
            plan=plan, 
            validation_rule=validation_rule, 
            available_actions=available_actions, 
            env_actions=env_actions, 
            goal=goal, 
            logger=logger, 
            do_replan=do_replan, 
            max_replan_count=5
        )
    
        save_json(result_states, os.path.join(experiment_path, task["name"] + f"_{exp_run}.json"))
        if save_video:
            display_images(
                images=env.cache_video[::13], 
                save_path=os.path.join(experiment_path, task["name"] + f"_{exp_run}.gif"), 
                do_display=False
            )

In [ ]:
images_to_use = 13
images_step = 13
display_images(
    images=env.cache_video[-(images_to_use-1)*images_step-1::images_step], 
    # save_path=os.path.join(experiment_path, task["name"] + f"_{exp_run}.gif"), 
    do_display=True
)

In [ ]:
replanner = Replanner(experiment_name, images_to_use=images_to_use, images_step=images_step, model_name="gpt-4o")


In [ ]:
goal = "put blocks in bowls with same color"
success_actions = ["locate('blue block')", "pick('blue block')", "locate('blue bowl')", "place('blue bowl')", "locate('red block')", "pick('red block')", "locate('red bowl')"]
current_plan = ["place('red bowl')", "done()"]

obj_list = task2["obj_list"]
env_actions = ["locate", "pick", "place", "done"]
available_actions = list(set([
    f"{action}('{obj}')" if action != 'done' else "done()"
    for action in env_actions
    for obj in obj_list
]))
current_images = "~/work/pybullet/Pybullet/saved_images/unnamed.png"


replanner.replan(goal, success_actions, current_plan, available_actions, current_images)

In [9]:
from llserver.utils.handler import UniserverHandler

In [ ]:
handler = UniserverHandler(port=8000)
prompt = """put blocks in bowls with same color###["locate(\'blue block\')", "pick(\'blue block\')", "locate(\'blue bowl\')", "place_on_top_of(\'blue bowl\')", "locate(\'red block\')", "pick(\'red block\')", "locate(\'red bowl\')"]###["place_on_top_of(\'red bowl\')", \'done()\']###["place_on_top_of(\'blue block\')", "pick(\'blue bowl\')", "locate(\'red block\')", \'done()\', "pick(\'red bowl\')", "locate(\'blue block\')", "pick(\'red block\')", "place_on_top_of(\'red block\')", "place_on_top_of(\'blue bowl\')", "place_on_top_of(\'red bowl\')", "pick(\'blue block\')", "locate(\'red bowl\')", "locate(\'blue bowl\')"]"""
dirs = ["~/work/meta_world/llserver/data/lera_test_replanner/0.png"]

put_response = handler.put_task(model_id="949c6169-db8e-4772-9d6f-5440a698cc2d", prompt=prompt, image_paths=dirs)
task_id = put_response["task_id"]["task_id"]

In [ ]:


status = ""
# initial wait time is large because the model 
# definitely needs to process the request
wait_time = 10
while status != "completed":
    time.sleep(wait_time)
    result_response = self.handler.get_task_result(model_id=self.model_id, task_id=task_id)
    status = result_response.get("status")
    print(f"Status: {status}")
    
    # after the first request, the wait time is smaller
    # initial wait time should not be divisible by new
    # this is made to avoid queues
    wait_time = 3

if status == "not found":
    raise Exception("Task failed: Task not found")

predicted_actions = result_response.get("result")
actions_list = predicted_actions.strip().strip("\n").strip('[]').split(', ')

In [ ]:
predicted_actions = ["locate('green block')", "pick('green block')", "locate('red block')", "place_on_top_of('red block')", "done()"]
actions_list = predicted_actions.strip('[]').split(', ')
actions_list

In [ ]:
result_states[2]["inputs"]["plan"]


In [ ]:
["locate('blue block')", "pick('blue block')", "locate('red bowl')", "place_on_top_of('red bowl')", "locate('red block')", "pick('red block')", "locate('red bowl')", "place_on_top_of('red bowl')", "locate('green block')", "pick('green block')", "locate('red bowl')", "place_on_top_of('red bowl')", 'done()]\n']

In [ ]:
replan_plan = replanning(goal, success_actions, current_plan, available_actions, current_image)
replan_plan


In [ ]:
prompt = f"""{goal}###{[]}###{result_states[2]["inputs"]["plan"]}###{available_actions}""".replace("place", "place_on_top_of") 
# save_image(current_image, f"image")

put_task_response = put_task(
    image_paths=[
        "~/work/pybullet/Pybullet/saved_images/image.png",
    ], 
    prompt=prompt
)

task_id = put_task_response.get("task_id")

status = ""
while status != "completed":
    result_response = get_task_result(task_id)
    status = result_response.get("status")
    print(f"Status: {status}")
    time.sleep(1)

if status == "not found":
    raise Exception("Task failed: Task not found")

predicted_actions = result_response.get("result")
actions_list = predicted_actions.strip('[]').split(', ')
actions_list

In [7]:
# result_states = run_experiment(env, OBJ_LIST, PLAN, VALIDATION_RULE, AVAILABLE_ACTIONS, ENV_ACTIONS, GOAL, logger, do_replan=False, max_replan_count=5)

In [109]:
# new_plan = ['"locate(\'red block\')"', '"pick(\'red block\')"', '"locate(\'blue bowl\')"', '"place_on_top_of(\'blue bowl\')"', '"locate(\'red block\')"', '"pick(\'red block\')"', '"locate(\'red bowl\')"', '"place_on_top_of(\'red bowl\')"', '"done()"']
# new_plan2 = ['"locate(\'red bowl\')"', '"place_on_top_of(\'red bowl\')"', '"pick(\'brown block\')"', '"place_on_top_of(\'cyan block\')"', "'done()'"]
# new_plan3 = ['"locate(\'cyan block\')"', '"pick(\'cyan block\')"', '"locate(\'red bowl\')"', '"place_on_top_of(\'cyan block\')"', '"locate(\'brown block\')"', '"pick(\'brown block\')"', '"locate(\'red bowl\')"', '"place_on_top_of(\'brown block\')"', "'done()'"]
# new_plan4 = ['"locate(\'brown block\')"', '"pick(\'brown block\')"', '"place_on_top_of(\'cyan block\')"', "'done()'"]

# good_plan = ['"locate(\'cyan block\')"', '"pick(\'cyan block\')"', '"locate(\'red bowl\')"', '"place_on_top_of(\'red bowl\')"', '"locate(\'brown block\')"', '"pick(\'brown block\')"', '"place_on_top_of(\'brown block\')"', '"done()"']

# # ['"locate(\'red block\')"', '"pick(\'red block\')"', '"locate(\'blue bowl\')"', '"place_on_top_of(\'blue bowl\')"', '"locate(\'red block\')"', '"pick(\'red block\')"', '"locate(\'red bowl\')"', '"place_on_top_of(\'red bowl\')"', '"done()"']
# validate_plan(new_plan, tmp["inputs"]["available_actions"])
# tmp = {'inputs': {'goal': 'put blocks in bowls with same color', 'obj_list': ['blue block', 'red block', 'blue bowl', 'red bowl'], 'plan': "locate('blue block')\npick('blue block')\nlocate('blue bowl')\nplace('blue bowl')\nlocate('red block')\npick('red block')\nlocate('red bowl')\nplace('red bowl')\ndone()", 'validation_rule': [('blue block', 'blue bowl'), ('red block', 'red bowl')], 'available_actions': ["locate('red block')", "place('red block')", "locate('red bowl')", "locate('blue block')", "locate('blue bowl')", 'done()', "pick('red block')", "place('red bowl')", "pick('blue bowl')", "place('blue bowl')", "pick('blue block')", "place('blue block')", "pick('red bowl')"]}, 'action_idx': 7, 'action': "place('red bowl')", 'action_name': 'place', 'object_name': 'red bowl', 'success': False, 'done': False, 'last_action_success': False, 'validation_rule': [('blue block', 'blue bowl'), ('red block', 'red bowl')], 'gripper_object': None, 'plan_actions': ["locate('blue block')", "pick('blue block')", "locate('blue bowl')", "place('blue bowl')", "locate('red block')", "pick('red block')", "locate('red bowl')", "place('red bowl')", 'done()'], 'replan_count': 0, 'was_replanning_successful': True}


In [ ]:
display_images(env.cache_video[::13])